In [1]:
from appgeopy import *
from my_packages import *

In [2]:
files = glob(os.path.join("calib_diffdisp/", "20251016*.csv"))
files

['calib_diffdisp\\20251016_GTWR_InputData_DiffDisp_Layer_1.csv',
 'calib_diffdisp\\20251016_GTWR_InputData_DiffDisp_Layer_2.csv',
 'calib_diffdisp\\20251016_GTWR_InputData_DiffDisp_Layer_3.csv',
 'calib_diffdisp\\20251016_GTWR_InputData_DiffDisp_Layer_4.csv',
 'calib_diffdisp\\20251016_GTWR_InputData_DiffDisp_Layer_All.csv']

In [3]:
from esda import Moran, Moran_Local

# PySAL
from libpysal.weights import KNN, DistanceBand, lag_spatial

# Optional plotting
# from splot.esda import moran_scatterplot, lisa_cluster


# 3) Prepare a function to build weights (choose KNN or DistanceBand)
def build_weights(points_gdf, method="knn", k=5, dist_band=None):
    coords = np.column_stack([points_gdf.geometry.x, points_gdf.geometry.y])
    if method == "knn":
        w = KNN.from_array(coords, k=k)
    else:
        # distance-band; set threshold to ensure connectivity (e.g., max nearest-neighbor distance)
        if dist_band is None:
            # compute nearest-neighbor distances to pick a safe threshold
            from sklearn.neighbors import NearestNeighbors

            nbrs = NearestNeighbors(n_neighbors=2).fit(coords)
            dists, _ = nbrs.kneighbors(coords)
            # dists[:,1] is NN distance; take its max and add a small buffer
            dist_band = float(dists[:, 1].max() * 1.05)
        w = DistanceBand.from_array(
            coords, threshold=dist_band, silence_warnings=True
        )
    w.transform = "R"  # row-standardize
    return w

In [4]:
time_col = "monthly"
var_col = "Layer_1"
x_col, y_col = "X_TWD97", "Y_TWD97"


f = files[0]
df = pd.read_csv(f)


gdf = geospatial.convert_to_geodata(
    df=df, xcoord_col=x_col, ycoord_col=y_col, crs_epsg="EPSG:3826"
)

unique_times = np.sort(gdf[time_col].unique())

# Loop over time steps and compute Moran’s I (global and local)
results = []
lisa_all = {}  # optional store local moran for each time

t = unique_times[0]
g_t = gdf[gdf[time_col] == t].copy()

# Build weights (KNN k=5 is a good starting point for 29 stations)
w = build_weights(g_t, method="knn", k=5)

y = g_t[var_col].values.astype(float)
# Optionally standardize (Moran internally centers; not required)
moran = Moran(y, w, permutations=999)

results.append(
    {
        "var": "DIFFDISP_" + var_col,
        "time": t,
        "n": len(g_t),
        "I": moran.I,
        "I_expected": moran.EI,
        "var_I": moran.VI_norm,
        "z_norm": moran.z_norm,
        "p_norm": moran.p_norm,
        "p_perm": moran.p_sim,
    }
)

# Optional: Local Moran’s I (LISA)
lisa = Moran_Local(y, w, permutations=999)
lisa_all[t] = {
    "Is": lisa.Is,  # local I values
    "q": lisa.q,  # cluster quadrant (1: HH, 2: LH, 3: LL, 4: HL)
    "p_sim": lisa.p_sim,  # permutation p-values
    "sig": lisa.p_sim < 0.05,  # significance mask
}

res_df = pd.DataFrame(results).sort_values("time")

In [19]:
res_df

,var,time,n,I,I_expected,var_I,z_norm,p_norm,p_perm
0,DIFFDISP_Layer_1,1,29,0.231466,-0.035714,0.009253,2.777527,0.005477,0.009


In [11]:
lisa_all[1]

{'Is': array([-5.15393754e-01,  1.01914670e+00,  2.19117183e+00,  2.34039793e-01,
         1.27873742e-03,  5.50164759e-03,  1.29860712e-01,  1.27873742e-03,
         1.67866904e-01,  4.19050787e-01,  1.03815942e-01, -1.00989597e+00,
        -5.24526785e-02, -3.33080364e-01, -1.06092978e+00,  2.58210837e-01,
         6.24241765e-01, -8.12218763e-02, -1.24722203e-01,  2.65949349e-01,
         2.44127489e-01,  1.18360128e-02,  1.38544262e-01,  6.93682053e-01,
         4.70084599e-01, -4.37259444e-01,  2.74198649e+00,  6.31964541e-03,
         3.68016974e-01]),
 'q': array([4, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 4, 4, 4, 4, 3, 1, 4, 2, 1, 1, 1,
        3, 3, 1, 4, 3, 1, 1]),
 'p_sim': array([0.004, 0.023, 0.025, 0.023, 0.495, 0.458, 0.222, 0.497, 0.128,
        0.033, 0.31 , 0.002, 0.351, 0.033, 0.002, 0.064, 0.024, 0.361,
        0.194, 0.005, 0.323, 0.002, 0.173, 0.106, 0.012, 0.008, 0.006,
        0.168, 0.06 ]),
 'sig': array([ True,  True,  True,  True, False, False, False, False, False,
 

In [16]:
g_t.columns

Index(['STATION', 'X_TWD97', 'Y_TWD97', 'monthly', 'time', 'Layer_1',
       'DIFFDISP', 'geometry'],
      dtype='object')